### 데이터셋 정의
* 이미지: torchvision.tv_tensors.Image의 모양 [3, H, W], 순수 텐서 또는 PIL 이미지 크기(H, W)
* 대상: 다음 필드를 포함하는 딕셔너리
* 박스, torchvision.tv_tensors.BoundingBoxes의 모양 [N, 4]
* 0에서 W까지, 0에서 H까지 범위의 [x0, y0, x1, y1] 형식의 N 바운딩 박스의 좌표
* 레이블, 정수 토치.텐서의 모양 [N]: 각 바운딩 박스에 대한 레이블.
* 0은 항상 배경 클래스를 나타냅니다.
* image_id, int: 이미지 식별자. 이 식별자는 데이터셋의 모든 이미지 간에 고유해야 하며, 평가 중에 사용됩니다.
* rea, float torch.Tensor의 모양 [N]: 바운딩 박스의 면적. 이 값은 COCO 메트릭으로 평가하는 동안 작은 박스(small), 중간 박스(medium), 큰 박스(large) 간에 메트릭 점수를 구분하는 데 사용됩니다.
* iscrowd, uint8 torch.Tensor의 모양 [N]: iscrowd=True인 인스턴스는 평가 중에 무시됩니다.
* (선택 사항) masks, torchvision.tv_tensors.Mask의 모양 [N, H, W]: 각 오브젝트에 대한 세그먼트 마스크

In [10]:
import os
import torch

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
import torchvision.transforms as transforms
import torch.utils.data


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # 모든 이미지 파일을 정렬되어 분류하여 불러온다.
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # 이미지와 마스크를 로드
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = read_image(img_path)
        mask = read_image(mask_path)
        # 인스턴스를 다른 색상으로 인코딩
        obj_ids = torch.unique(mask)
        # 처음 id는 배경이므로 제거
        obj_ids = obj_ids[1:]
        num_objs = len(obj_ids)

        # 색상 인코딩 마스크를 바이너리 마스크 집합으로 분할
        masks = (mask == obj_ids[:, None, None]).to(dtype=torch.uint8)

        # 각 마스크의 바운딩 박스좌표 가져오기
        boxes = masks_to_boxes(masks)

        # 클래스가 하나 뿐이다.
        labels = torch.ones((num_objs,), dtype=torch.int64)

        image_id = idx
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # 모든 인스턴스가 군중이 아니라고 가정한다.
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # sample과 target을 torchvision의 tv_tensors로 래핑
        img = tv_tensors.Image(img)

        target = {}
        target["boxes"] = tv_tensors.BoundingBoxes(boxes, format="XYXY", canvas_size=F.get_size(img))
        target["masks"] = tv_tensors.Mask(masks)
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

### 모델 정의하기
* Faster R-CNN을 기반으로 하는 Mask R-CNN을 사용
* Mask R-CNN은 각 인스턴스에 대한 세그멘테이션 마스크도 예측하는 Faster R-CNN에 추가 브렌치(분기)를 추가
* TorchVision Model Zoo에서 사용 가능한 모델 중 하나를 수정해야 하는 일반적인 상황은 두 가지
* 첫 번째는 사전 학습된 모델에서 시작하여 마지막 레이어만 파인튜닝하려는 경우
* 다른 하나는 모델의 백본(backbone)을 다른 모델로 교체하려는 경우

#### 사전 학습된 모델에서 파인튜닝하기

In [1]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# COCO에서 사전학습된 모델을 로드
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

# 분류기(classifier) 사용자가 정의한 num_classes를 가진 새로운 것으로 교체
num_classes = 2  # 1 class (person) + background
# 분류기에 대한 입력 특징 수 가져오기
in_features = model.roi_heads.box_predictor.cls_score.in_features
# 사전 학습된 헤드를 새 헤드로 교체합니다.
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\008yo/.cache\torch\hub\checkpoints\fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:02<00:00, 57.4MB/s] 


#### 모델을 수정하여 다른 백본 추가하기

In [2]:
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# 분류를 위해 미리 학습된 모델을 로드하고 특징만 반환
backbone = torchvision.models.mobilenet_v2(weights="DEFAULT").features
# ``FasterRCNN``은 백본의 출력 채널 수를 알아야 합니다. mobilenet_v2의 경우 1280이므로 여기에 추가해야 한다.
backbone.out_channels = 1280

# RPN이 공간 위치당 5개의 서로 다른 크기와 3개의 서로 다른 종횡비(aspect ratios)를 가진 5 x 3 앵커를 생성하도록 만들어 봅시다. 각 피처 맵의 크기와 종횡비가 다를 수 있으므로 Tuple[Tuple[int]]을 사용합니다.
anchor_generator = AnchorGenerator(
    sizes=((32, 64, 128, 256, 512),),
    aspect_ratios=((0.5, 1.0, 2.0),)
)

# 관심 영역 자르기(crop)를 수행하는 데 사용할 피처 맵(feature map)과 리스케일링 후 자를(crop) 크기(size)를 정의해 보겠습니다. 백본이 텐서를 반환하는 경우 featmap_names는 [0]이 될 것으로 예상됩니다. 보다 일반적으로 백본은 ``OrderedDict[Tensor]``를 반환해야 하며, ``featmap_names``에서 어떤 피처 맵을 사용할지 선택할 수 있습니다.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(
    featmap_names=['0'],
    output_size=7,
    sampling_ratio=2,
)

# Faster-RCNN 모델 안에 구성 요소 넣기
model = FasterRCNN(
    backbone,
    num_classes=2,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler,
)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to C:\Users\008yo/.cache\torch\hub\checkpoints\mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 48.9MB/s]


#### 펜푸단 데이터셋에 대한 오브젝트 디텍션 및 인스턴스 세그멘테이션 모델
* 인스턴스 세그멘테이션 마스크도 계산해야 하므로 Mask R-CNN을 사용

In [3]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # COCO에서 사전 학습된 인스턴스 세그멘테이션 모델을 로드
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # 분류기에 대한 입력 기능 수 가져오기
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # 이제 마스크 분류기에 대한 입력 피처 수를 가져오기
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # 마스크 예측기를 새 예측기로 교체
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask,
        hidden_layer,
        num_classes,
    )

    return model

In [6]:
import os
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py")

1

* 데이터 증강/변환을 위한 몇 가지 헬퍼 함수를 작성

In [7]:
from torchvision.transforms import v2 as T


def get_transform(train):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))  # 랜덤 수평 플립
    transforms.append(T.ToDtype(torch.float, scale=True)) 
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)

### forward() 메서드 테스트
* 데이터 집합을 반복하기 전에 샘플 데이터에 대한 학습 및 추론 시간 동안 모델이 무엇을 기대하는지 확인

In [11]:
import utils
import os


model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
dataset = PennFudanDataset(os.getenv('HOME')+'/aiffel/pytorch_cv/data/PennFudanPed', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    num_workers=3,
    collate_fn=utils.collate_fn
)

# 학습 코드
images, targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets)  # 손실값과 디텍션 출력

print(output)

# 추론 코드
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)  # 예측 출력
print(predictions[0])

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

* 학습과 유효성 검사를 수행하는 메인 함수

In [12]:
from engine import train_one_epoch, evaluate

# GPU 또는 GPU를 사용할 수 없는 경우, CPU에서 훈련
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# 우리 데이터셋에는 배경과 사람, 두 개의 클래스만 있습니다.
num_classes = 2

# 데이터셋 및 정의된 변환 사용
dataset = PennFudanDataset(os.getenv('HOME')+'/aiffel/pytorch_cv/data/PennFudanPed', get_transform(train=True))
dataset_test = PennFudanDataset(os.getenv('HOME')+'/aiffel/pytorch_cv/data/PennFudanPed', get_transform(train=False))

# 데이터 집합을 학습 및 테스트 집합으로 분할
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# 학습 및 평가 데이터 로더 정의하기
data_loader = torch.utils.data.DataLoader(
    dataset,
    pin_memory=True,
    batch_size=2,
    shuffle=True,
    num_workers=3,
    collate_fn=utils.collate_fn
)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    pin_memory=True,
    batch_size=1,
    shuffle=False,
    num_workers=3,
    collate_fn=utils.collate_fn
)

# 헬퍼 함수를 사용하여 모델 가져오기
model = get_model_instance_segmentation(num_classes)

# 올바른 기기로 모델 이동
model.to(device)

# 옵티마이저 구성
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005
)

# 학습률 스케줄러
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.1
)

# 5 에포크 학습하기
num_epochs = 5

for epoch in range(num_epochs):
    # 1 에포크 학습마다, 10회 반복마다 출력한다.
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # 학습률 업데이트
    lr_scheduler.step()
    # 테스트 데이터셋으로 평가
    evaluate(model, data_loader_test, device=device)

print("That's it!")

ModuleNotFoundError: No module named 'engine'

In [13]:
import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks


image = read_image(os.getenv('HOME')+'/data/tv_image05.png')
eval_transform = get_transform(train=False)

model.eval()
with torch.no_grad():
    x = eval_transform(image)
    # RGBA -> RGB로 변환시켜서 장치에 넣어줍니다.
    x = x[:3, ...].to(device)
    predictions = model([x, ])
    pred = predictions[0]

image = (255.0 * (image - image.min()) / (image.max() - image.min())).to(torch.uint8)
image = image[:3, ...]
pred_labels = [f"pedestrian: {score:.3f}" for label, score in zip(pred["labels"], pred["scores"])]
pred_boxes = pred["boxes"].long()
output_image = draw_bounding_boxes(image, pred_boxes, pred_labels, colors="red")

masks = (pred["masks"] > 0.7).squeeze(1)
output_image = draw_segmentation_masks(output_image, masks, alpha=0.5, colors="blue")

plt.figure(figsize=(12, 12))
plt.imshow(output_image.permute(1, 2, 0))

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'